In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import sys
sys.path.append("../")
from constants import *

In [2]:
movies_df = pd.read_csv("../data/ml-25m/ml-25m/movies.csv")

In [3]:
len(movies_df)

62423

In [4]:
movies_df.tail(10)

,movieId,title,genres
62413,209145,Liberté (2019),Drama
62414,209147,The Carpet of Horror (1962),Crime|Horror
62415,209151,Mao Zedong 1949 (2019),(no genres listed)
62416,209153,Happy Flight (2008),Comedy|Drama
62417,209155,Santosh Subramaniam (2008),Action|Comedy|Romance
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)
62422,209171,Women of Devil's Island (1962),Action|Adventure|Drama


In [5]:
ratings_df = pd.read_csv("../data/ml-25m/ml-25m/ratings.csv")

In [6]:
len(ratings_df)

25000095

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [8]:
ratings_df.sort_values(by=["timestamp"], inplace=True)
grouped_ratings = ratings_df.groupby(by="userId").agg(list)

In [9]:
grouped_ratings.head(10)

,movieId,rating,timestamp
userId,,,
1,"[5952, 2012, 2011, 1653, 1250, 6539, 6377, 344...","[4.0, 2.5, 2.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, ...","[1147868053, 1147868068, 1147868079, 114786809..."
2,"[2797, 5952, 1080, 553, 653, 497, 1374, 1653, ...","[1.0, 5.0, 1.0, 2.0, 3.0, 4.0, 4.5, 4.5, 3.0, ...","[1141415509, 1141415528, 1141415532, 114141553..."
3,"[356, 593, 1270, 1, 480, 2571, 260, 318, 1196,...","[4.0, 4.0, 3.5, 4.0, 2.0, 4.0, 4.0, 4.0, 4.0, ...","[1439472199, 1439472203, 1439472211, 143947221..."
4,"[97913, 93510, 91630, 93840, 195159, 122914, 1...","[3.5, 4.0, 3.5, 4.5, 5.0, 3.0, 2.0, 3.5, 2.5, ...","[1573937091, 1573937096, 1573937103, 157393711..."
5,"[592, 590, 296, 150, 344, 153, 588, 595, 231, ...","[3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 4.0, ...","[830786155, 830786155, 830786155, 830786155, 8..."
6,"[2421, 1466, 161, 902, 858, 2815, 1183, 1704, ...","[3.0, 3.0, 2.0, 4.0, 5.0, 3.0, 2.0, 5.0, 5.0, ...","[945141530, 945141530, 945141530, 945141564, 9..."
7,"[590, 296, 592, 150, 153, 165, 344, 588, 595, ...","[3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 2.0, 4.0, 4.0, ...","[835444730, 835444730, 835444730, 835444730, 8..."
8,"[1672, 1617, 1777, 1721, 1704, 551, 903, 110, ...","[4.0, 5.0, 3.0, 4.0, 4.0, 2.0, 4.0, 5.0, 3.0, ...","[890489203, 890489203, 890489236, 890489263, 8..."
9,"[1073, 260, 1356, 805, 1210, 667, 1367, 61, 85...","[5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 3.0, ...","[859381992, 859382015, 859382042, 859382042, 8..."


In [10]:
movieIdMapping = {k:i+2 for i, k in enumerate(sorted(list(ratings_df.movieId.unique())))}

In [11]:
len(movieIdMapping)

59047

In [13]:
len(movies_df)

62423

In [14]:
max(movieIdMapping.keys())

209171

In [15]:
min(movieIdMapping.keys())

1

In [17]:
max(movieIdMapping.values())

59048

In [18]:
min(movieIdMapping.values())


2

In [19]:
ratings_df["movieId_mapped"] = ratings_df.movieId.map(movieIdMapping)

In [20]:
ratings_df.head()

,userId,movieId,rating,timestamp,movieId_mapped
326761,2262,21,3.0,789652009,22
326810,2262,1079,3.0,789652009,1054
326767,2262,47,5.0,789652009,48
15845015,102689,1,4.0,822873600,2
15845023,102689,39,5.0,822873600,40


In [21]:
ratings_df.movieId_mapped.max()

59048

In [22]:
ratings_df.movieId_mapped.min()

2

In [23]:
movies_df["movieId_mapped"] = movies_df.movieId.map(movieIdMapping)

In [25]:
movies_df.to_csv("../data/ml-25m/ml-25m/movies_mapped.csv", index = False)

In [26]:
ratings_df.to_csv("../data/ml-25m/ml-25m/ratings_mapped.csv", index = False)